In [56]:
import pandas as pd
import numpy as np

In [57]:
# Dataframes from Census-Data
census_column_keys = pd.read_csv("../shared_data_read_only/Census-Data/ACSST5Y2022.S2201-Column-Metadata.csv")
census_data = pd.read_csv("../shared_data_read_only/Census-Data/ACSST5Y2022.S2201-Data.csv", low_memory=False)
nonprofit_data = pd.read_csv("../shared_data_read_only/Census-Data/Nonprofit_Data_PA.csv")

In [58]:
# Dataframes from Education-Data
enrollment_demographics = pd.read_csv("../shared_data_read_only/Education-Data/2023-2024 Enrollment & Demographics.csv")
master_school_list = pd.read_csv("../shared_data_read_only/Education-Data/2023-2024 Master School List (20231003).csv")
college_matriculation = pd.read_csv("../shared_data_read_only/Education-Data/SDP_College_Matriculation_2021-2022.csv")
graduation_rates = pd.read_csv("../shared_data_read_only/Education-Data/SDP_Graduation_Rates_School_S_2023-02-15.csv")
school_metrics = pd.read_csv("../shared_data_read_only/Education-Data/SPREE_SY2122_School_Metric_Scores.csv")
student_ada = pd.read_csv("../shared_data_read_only/Education-Data/Student_ADA_Yearly.csv")

In [59]:
# Dataframes from Geography-Mapping
geography_map = pd.read_csv("../shared_data_read_only/Geography-Mapping/ZIP_TRACT_122023.csv")

In [60]:
#Clean and merge the census data

#check if NAME column contains "Pennsylvania"
#list(census_data.columns)
PA_census_data = census_data[census_data['NAME'].str.contains('Pennsylvania')]
print(PA_census_data.shape[0]) #this has 3446 because of multiple tracts

#remove the leading "1400000US" in front to extract the tract number
PA_census_data.loc[:, 'GEO_ID'] = PA_census_data['GEO_ID'].str.replace('1400000US', '')
PA_census_renamed = PA_census_data.rename({'GEO_ID' : 'TRACT'}, axis=1)

#filter down the geography map
PA_geography_map = geography_map[geography_map['USPS_ZIP_PREF_STATE'].str.contains('PA')]
print(PA_geography_map.shape[0]) #8907 values, greater than our 3446 so not every tract is represented


3446
8907


In [61]:
#convert TRACT numbers to strings
PA_census_renamed['TRACT'] = PA_census_renamed['TRACT'].astype(str)
PA_geography_map.loc[:, 'TRACT'] = PA_geography_map['TRACT'].astype(str)

#map the tract number to the zip code
merged_census = pd.merge(PA_census_renamed, PA_geography_map, on='TRACT', how='left')

#move zip to be 1st column
columns = ['ZIP'] + [col for col in merged_census.columns if col != 'ZIP']

#reassign the column order
merged_census = merged_census[columns] #contains census data and geographical information so far

# sort by zip code so we can see what is happening and how to collapse
merged_census.sort_values(by='ZIP', ascending=True, inplace=True)

print(merged_census.shape[0])
print(merged_census.head(10))

#convert object types to numeric data
object_cols = merged_census.select_dtypes(include=['object']).columns

for col in object_cols:
    merged_census[col] = pd.to_numeric(merged_census[col], errors='coerce')

#get the numeric columns only
numeric_cols = merged_census.select_dtypes(include=[np.float64, np.float32]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['ZIP', 'Unnamed: 458']]

# make sure zip is not part of the averaging columns
print("Columns for averaging:", numeric_cols)


# collpase by averageing out the values in each column
collapsed_data = merged_census.groupby('ZIP')[numeric_cols].median().reset_index()

print(collapsed_data.shape[0])
print(collapsed_data.head(10))

8908
          ZIP        TRACT                                               NAME  \
1101  15001.0  42007604901  Census Tract 6049.01; Beaver County; Pennsylvania   
1064  15001.0  42007602900     Census Tract 6029; Beaver County; Pennsylvania   
1068  15001.0  42007603000     Census Tract 6030; Beaver County; Pennsylvania   
1073  15001.0  42007603202  Census Tract 6032.02; Beaver County; Pennsylvania   
1128  15001.0  42007605500     Census Tract 6055; Beaver County; Pennsylvania   
1104  15001.0  42007605001  Census Tract 6050.01; Beaver County; Pennsylvania   
1103  15001.0  42007604902  Census Tract 6049.02; Beaver County; Pennsylvania   
1107  15001.0  42007605002  Census Tract 6050.02; Beaver County; Pennsylvania   
1133  15001.0  42007605700     Census Tract 6057; Beaver County; Pennsylvania   
1099  15001.0  42007604800     Census Tract 6048; Beaver County; Pennsylvania   

     S2201_C01_001E S2201_C01_001M S2201_C01_002E S2201_C01_002M  \
1101           1636            141 

In [75]:
merged_census.head(5)

,ZIP,TRACT,NAME,S2201_C01_001E,S2201_C01_001M,S2201_C01_002E,S2201_C01_002M,S2201_C01_003E,S2201_C01_003M,S2201_C01_004E,...,S2201_C06_037M,S2201_C06_038E,S2201_C06_038M,Unnamed: 458,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
1101,15001.0,42007604901,NaN,1636,141,858,146,778,169,840,...,8.9,69.0,8.8,NaN,NaN,NaN,0.109431,0.092672,0.102778,0.108464
1064,15001.0,42007602900,NaN,1026,145,527,114,499,89,641,...,6.7,54.8,8.3,NaN,NaN,NaN,0.009269,0.000000,0.005556,0.008733
1068,15001.0,42007603000,NaN,1075,74,535,85,540,82,643,...,5.8,62.5,7.8,NaN,NaN,NaN,0.062855,0.029095,0.005556,0.060054
1073,15001.0,42007603202,NaN,2952,252,1424,249,1528,205,1867,...,8.4,53.4,7.9,NaN,NaN,NaN,0.108678,0.066810,0.150000,0.107385
1128,15001.0,42007605500,NaN,2191,225,1069,237,1122,200,1248,...,10.4,54.2,11.5,NaN,NaN,NaN,0.030298,0.050647,0.061111,0.031914


In [63]:
# Dropping unnecessary rows from enrollment_demographics dataframe.
updated_enrollment_demographics = enrollment_demographics.loc[enrollment_demographics['GradeLevel'] == 'All Grades']
print(len(updated_enrollment_demographics))

331


In [64]:
updated_student_ada = student_ada.loc[student_ada['School Year'] == '2021-2022']
print(len(updated_student_ada))

214


In [65]:
# Dropping unnecessary rows from updated_graduation_rates dataframe.
print(len(graduation_rates))
graduation_rates['score'] = pd.to_numeric(graduation_rates['score'], errors='coerce')
updated_cleaned_data = graduation_rates.dropna(subset=['score'])
updated_graduation_rates = updated_cleaned_data.groupby(['schoolid_ulcs', 'schoolname', 'sector', 'rate_type', 'group', 'subgroup'])['score'].mean().reset_index()
updated_graduation_rates.rename(columns={'score': 'average_score'}, inplace=True)
updated_graduation_rates = updated_graduation_rates.loc[updated_graduation_rates['group'] == 'All Students']
updated_graduation_rates = updated_graduation_rates.loc[updated_graduation_rates['rate_type'] == '4-Year Graduation Rate']
print(len(updated_graduation_rates))
updated_graduation_rates.head(5)

26163
95


,schoolid_ulcs,schoolname,sector,rate_type,group,subgroup,average_score
0,1010,John Bartram High School,District,4-Year Graduation Rate,All Students,All Students,64.932222
36,1020,West Philadelphia High School,District,4-Year Graduation Rate,All Students,All Students,64.043333
61,1030,High School of the Future,District,4-Year Graduation Rate,All Students,All Students,80.133333
88,1050,Paul Robeson High School for Human Services,District,4-Year Graduation Rate,All Students,All Students,94.727778
118,1080,University City High School,District,4-Year Graduation Rate,All Students,All Students,12.000000


In [67]:
# converting the names of schools to lowercase
updated_enrollment_demographics['SchoolName'] = updated_enrollment_demographics['SchoolName'].str.lower()
updated_enrollment_demographics.rename(columns={'SchoolName': 'School Name'}, inplace=True)
master_school_list['School Name (ULCS)'] = master_school_list['School Name (ULCS)'].str.lower()
master_school_list.rename(columns={'School Name (ULCS)': 'School Name'}, inplace=True)
college_matriculation['School Name'] = college_matriculation['School Name'].str.lower()
updated_student_ada['School Name'] = updated_student_ada['School Name'].str.lower()
updated_graduation_rates.rename(columns={'schoolname': 'School Name'}, inplace=True)
updated_graduation_rates['School Name'] = college_matriculation['School Name'].str.lower()
updated_graduation_rates.head(5)

/tmp/ipykernel_92990/1720002524.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_enrollment_demographics['SchoolName'] = updated_enrollment_demographics['SchoolName'].str.lower()
/tmp/ipykernel_92990/1720002524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_enrollment_demographics.rename(columns={'SchoolName': 'School Name'}, inplace=True)
/tmp/ipykernel_92990/1720002524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

,schoolid_ulcs,School Name,sector,rate_type,group,subgroup,average_score
0,1010,john bartram high school,District,4-Year Graduation Rate,All Students,All Students,64.932222
36,1020,central high school,District,4-Year Graduation Rate,All Students,All Students,64.043333
61,1030,NaN,District,4-Year Graduation Rate,All Students,All Students,80.133333
88,1050,NaN,District,4-Year Graduation Rate,All Students,All Students,94.727778
118,1080,NaN,District,4-Year Graduation Rate,All Students,All Students,12.000000


In [68]:
# Merging the four dataframes from the education-data
education_merged_df = pd.merge(updated_enrollment_demographics, master_school_list, on='School Name', how='outer')
education_merged_df = pd.merge(education_merged_df, college_matriculation, on='School Name', how='outer')
education_merged_df = pd.merge(education_merged_df, updated_student_ada, on='School Name', how='outer')
education_merged_df = pd.merge(education_merged_df, updated_graduation_rates, on='School Name', how='outer')
education_merged_df.head(10)

,SchoolYear,Sector_x,SubSector,ulcscode,School Name,LearningNetwork,GradeLevel,StudentEnrollment,ELCount,ELPCT,...,School Year_y,Sector_y,ULCS Code,Average Daily Attendance (YTD),schoolid_ulcs,sector,rate_type,group,subgroup,average_score
0,2023-2024,District,District,1010.0,john bartram high school,Network 13,All Grades,526.0,114.0,21.67,...,2021-2022,District,1010.0,76.70,1010.0,District,4-Year Graduation Rate,All Students,All Students,64.932222
1,2023-2024,District,District,1020.0,west philadelphia high school,Network 13,All Grades,601.0,30.0,4.99,...,2021-2022,District,1020.0,85.87,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-2024,District,District,1030.0,high school of the future,Innovation,All Grades,560.0,11.0,1.96,...,2021-2022,District,1030.0,83.85,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-2024,District,District,1050.0,paul robeson high school for human services,Network 1,All Grades,291.0,15.0,5.15,...,2021-2022,District,1050.0,92.37,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-2024,District,District,1100.0,william l. sayre high school,Network 13,All Grades,426.0,16.0,3.76,...,2021-2022,District,1100.0,81.19,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-2024,District,District,1130.0,william t. tilden school,Network 10,All Grades,281.0,59.0,21.00,...,2021-2022,District,1130.0,88.33,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-2024,District,District,1190.0,motivation high school,Network 1,All Grades,239.0,24.0,10.04,...,2021-2022,District,1190.0,92.34,NaN,NaN,NaN,NaN,NaN,NaN
7,2023-2024,District,District,1200.0,john barry school,Network 5,All Grades,493.0,9.0,1.83,...,2021-2022,District,1200.0,80.45,NaN,NaN,NaN,NaN,NaN,NaN
8,2023-2024,District,District,1230.0,william c. bryant school,Acceleration,All Grades,341.0,18.0,5.28,...,2021-2022,District,1230.0,80.42,NaN,NaN,NaN,NaN,NaN,NaN
9,2023-2024,District,District,1250.0,joseph w. catharine school,Network 12,All Grades,398.0,98.0,24.62,...,2021-2022,District,1250.0,87.39,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
#Filter down to just Pennslyvania
print(len(education_merged_df))
print(education_merged_df['School Name'].nunique())
for column in education_merged_df.columns:
    print(column)
# print(for column in education_merged_df.columns)

580
487
SchoolYear
Sector_x
SubSector
ulcscode
School Name
LearningNetwork
GradeLevel
StudentEnrollment
ELCount
ELPCT
NotELCount
NotELPCT
IEPCount
IEPPCT
NotIEPCount
NotIEPPCT
FemaleCount
FemalePCT
MaleCount
MalePCT
AmericanIndianCount
AmericanIndianPCT
AsianCount
AsianPCT
BlackAfricanAmericanCount
BlackAfricanAmericanPCT
HispanicCount
HispanicPCT
MultiRaceCount
MultiRacePCT
PacificIslanderCount
PacificIslanderPCT
WhiteCount
WhitePCT
CEPEconomicallyDisadvantagedRate
PA Code
NSC Code
NCES Code
AUN Code
ULCS Code_x
SRC School ID
Publication Name
Publication Name Alpha List
Abbreviated Name
Year Opened
School Level
Admission Type
Current Grade Span Served
Grade Span at Scale
Phasing-In
Phasing-Out
Governance
Management Organization
School Reporting Category
Alternate Education Type
Major Intervention
Major Intervention Year
Community School Cohort
CTE Status
Title I Designation
Federal Accountability Designation
City Council District
GPS Location
Multiple Addresses
Street Address
City
Sta

In [70]:
#Filter data down to radius of 15 miles (see list of generate zip codes)
print(education_merged_df.head(5))

  SchoolYear  Sector_x SubSector  ulcscode  \
0  2023-2024  District  District    1010.0   
1  2023-2024  District  District    1020.0   
2  2023-2024  District  District    1030.0   
3  2023-2024  District  District    1050.0   
4  2023-2024  District  District    1100.0   

                                   School Name LearningNetwork  GradeLevel  \
0                     john bartram high school      Network 13  All Grades   
1                west philadelphia high school      Network 13  All Grades   
2                    high school of the future      Innovation  All Grades   
3  paul robeson high school for human services       Network 1  All Grades   
4                 william l. sayre high school      Network 13  All Grades   

   StudentEnrollment  ELCount  ELPCT  ...  School Year_y  Sector_y  ULCS Code  \
0              526.0    114.0  21.67  ...      2021-2022  District     1010.0   
1              601.0     30.0   4.99  ...      2021-2022  District     1020.0   
2          

In [71]:
columns_to_keep = ["ulcscode", "School Name", "StudentEnrollment", "FemalePCT", "MalePCT", "AmericanIndianPCT", "AsianPCT", "BlackAfricanAmericanPCT", 
                     "HispanicPCT", "MultiRacePCT", "PacificIslanderPCT", "WhitePCT", "CEPEconomicallyDisadvantagedRate", "PA Code", 
                     "NSC Code", "NCES Code", "AUN Code", "ULCS Code_x", "SRC School ID", "City", "State", "Zip Code", "Number of Graduates", 
                     "First-Fall Matriculation Rate", "Average Daily Attendance (YTD)", "average_score"]
education_merged_df = education_merged_df[columns_to_keep]
education_merged_df = education_merged_df.dropna(subset=['Zip Code'])
education_merged_df.head(5)

,ulcscode,School Name,StudentEnrollment,FemalePCT,MalePCT,AmericanIndianPCT,AsianPCT,BlackAfricanAmericanPCT,HispanicPCT,MultiRacePCT,...,AUN Code,ULCS Code_x,SRC School ID,City,State,Zip Code,Number of Graduates,First-Fall Matriculation Rate,Average Daily Attendance (YTD),average_score
0,1010.0,john bartram high school,526.0,45.82,53.99,0.00,3.42,77.00,16.35,3.04,...,126515001.0,1010.0,101,PHILADELPHIA,PA,19142.0,120.0,25.83,76.70,64.932222
1,1020.0,west philadelphia high school,601.0,45.76,54.24,0.50,1.00,88.19,4.33,4.49,...,126515001.0,1020.0,102,PHILADELPHIA,PA,19139.0,81.0,35.80,85.87,NaN
2,1030.0,high school of the future,560.0,49.46,50.36,0.71,1.07,92.32,3.39,1.43,...,126515001.0,1030.0,103,PHILADELPHIA,PA,19104.0,133.0,35.34,83.85,NaN
6,1190.0,motivation high school,239.0,47.70,51.88,0.42,0.84,92.47,2.93,2.51,...,126515001.0,1190.0,119,PHILADELPHIA,PA,19143.0,79.0,44.30,92.34,NaN
8,1230.0,william c. bryant school,341.0,53.37,46.63,0.29,1.76,89.74,5.87,1.47,...,126515001.0,1230.0,123,PHILADELPHIA,PA,19143.0,NaN,NaN,80.42,NaN


In [72]:
columns_to_keep = [ "School Name", "StudentEnrollment", "AsianPCT", "BlackAfricanAmericanPCT", 
                     "HispanicPCT", "WhitePCT", 
                       "City", "State", "Zip Code", "Number of Graduates", "Average Daily Attendance (YTD)", "average_score"]
education_merged_df = education_merged_df[columns_to_keep]
education_merged_df = education_merged_df.dropna(subset=['Zip Code'])
education_merged_df.head(10)

,School Name,StudentEnrollment,AsianPCT,BlackAfricanAmericanPCT,HispanicPCT,WhitePCT,City,State,Zip Code,Number of Graduates,Average Daily Attendance (YTD),average_score
0,john bartram high school,526.0,3.42,77.00,16.35,0.19,PHILADELPHIA,PA,19142.0,120.0,76.70,64.932222
1,west philadelphia high school,601.0,1.00,88.19,4.33,1.00,PHILADELPHIA,PA,19139.0,81.0,85.87,NaN
2,high school of the future,560.0,1.07,92.32,3.39,1.07,PHILADELPHIA,PA,19104.0,133.0,83.85,NaN
6,motivation high school,239.0,0.84,92.47,2.93,0.84,PHILADELPHIA,PA,19143.0,79.0,92.34,NaN
8,william c. bryant school,341.0,1.76,89.74,5.87,0.88,PHILADELPHIA,PA,19143.0,NaN,80.42,NaN
9,joseph w. catharine school,398.0,6.28,66.33,18.84,1.26,PHILADELPHIA,PA,19142.0,NaN,87.39,NaN
10,benjamin b. comegys school,267.0,0.75,93.26,3.37,1.12,PHILADELPHIA,PA,19143.0,NaN,82.53,NaN
12,andrew hamilton school,354.0,0.85,93.22,3.95,0.28,PHILADELPHIA,PA,19139.0,NaN,81.56,NaN
14,henry c. lea school,461.0,17.14,58.79,7.16,12.36,PHILADELPHIA,PA,19139.0,NaN,90.51,NaN
15,william c. longstreth school,248.0,0.00,94.76,3.23,0.81,PHILADELPHIA,PA,19143.0,NaN,84.69,NaN


In [73]:
# filtering only the zip codes inside 15 miles of radius of 19140
import json
file_path = '../Team-7/zipcodes_within_radius.json'
with open(file_path, 'r') as file:
    data = json.load(file)

zip_codes_list = [int(item['zip_code']) for item in data['zip_codes']]
education_merged_df = education_merged_df[education_merged_df['Zip Code'].isin(zip_codes_list)]
print(len(education_merged_df))

329


In [74]:
# finding the data of the 19140 zip code
row_19140 = education_merged_df[education_merged_df['Zip Code'] == '19140']
print(row_19140)

Empty DataFrame
Columns: [School Name, StudentEnrollment, AsianPCT, BlackAfricanAmericanPCT, HispanicPCT, WhitePCT, City, State, Zip Code, Number of Graduates, Average Daily Attendance (YTD), average_score]
Index: []


In [43]:
#Adding in educational data
#master_school_list_renamed = master_school_list.rename({'Zip Code' : 'ZIP'}, axis=1)
#master_school_list_renamed.head(10)
#master_school_list_renamed.

#merged_school_info = pd.merge(master_school_list, merged_census, on='ZIP'



In [ ]:
#Filter down to just Pennslyvania

In [ ]:
#Filter down the number of parameters in our data (some of these features are not necessarily the most important!)




In [ ]:
#Average the schools within each zip code



In [ ]:
#Filter data down to radius of 15 miles (see list of generate zip codes)